In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm

tickers = ['AAPL', 'TSLA', 'NVDA', 'META', 'AMZN', 'GOOG']



prices = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Close']

returns = prices.pct_change().dropna()
spy = yf.download('SPY', start=returns.index[0], end=returns.index[-1])['Close']
spy_returns = spy.pct_change().dropna()
returns.cumsum().plot(title="Ticker returns")

# Applying a strategy and obtaining returns
A rank demeaned strategy using a 20 day moving average window for the 6 tickers is implemented after which the strategy return streams along with the individual sharpe ratios are obtained.

In [ ]:
strat = {}
avg_ret = returns.rolling(20, min_periods=1).mean().rank(1)
avg_ret = avg_ret.subtract(avg_ret.mean(1), 0)
avg_ret = avg_ret.divide(avg_ret.abs().sum(1), 0)
strat = (avg_ret.shift()*returns)
strat.cumsum().plot(title="Strategy returns")

In [ ]:
sr = (strat.mean()/strat.std())*np.sqrt(252)
print("Sharpe ratios of individual strategies: ", sr)

# Calculating optimal weights using mean-variance optimization
Optimal weights are calculated using $w = \Sigma^{-1} \times \mu $.

Once the weights are assigned to each strategy, the combined returns stream is calculated by doing a dot product of the weights array and the return stream of the individual strategies

In [ ]:
def optimal_weights(sigma, mu):
    weights = np.linalg.inv(sigma) @ mu
    weights = weights / np.abs(weights).sum()
    return weights
sigma = strat.cov()
mu = strat.mean()
weights = optimal_weights(sigma, mu)

In [ ]:
combined_returns = strat @ weights


cum_individual = strat.cumsum()
cum_combined = combined_returns.cumsum()
cum_combined.plot()

In [ ]:
print("Sharpe ratio of combined portfolio: ", new_sharpe)

# Testing for alpha
Our combined strategy is regressed against the benchmark which is SPY or the S&P500.

In [ ]:
combined_returns = combined_returns.loc[spy_returns.index]
X = sm.add_constant(spy_returns)  
model = sm.OLS(combined_returns, X).fit()
print(model.summary())


print("Alpha (annualized):", model.params['const'] * 252)



# Max Drawdown Duration

In [ ]:
cum_returns = (1 + combined_returns).cumprod()
running_max = cum_returns.cummax()
drawdown = (cum_returns - running_max) / running_max
max_drawdown = drawdown.min()
print("Max Drawdown:", max_drawdown)